In [1]:
import sys
# Python 3.7 is required
assert sys.version_info >= (3,7)

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt


# Make sure that optimization is enabled
if not cv.useOptimized():
    cv.setUseOptimized(True)

cv.useOptimized()

True

1. Convert the code chunk found under section Divide an image into smaller patches using cropping into a function with the following signature:
crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color)

In [2]:
img = cv.imread('dog.jfif')
img_copy = img.copy()

def crop_grid(img, num_horizontal_grid, num_vertical_grid, line_color):
    height, width = img.shape[:2]
    M, N = int(height/num_horizontal_grid), int(width/num_vertical_grid)
    x1, y1 = 0, 0
    
    for y in range(0, height, M):
        for x in range(0, width, N):
            
            y1 = y + M   
            x1 = x + N
        
            if x1 >= width and y1 >= height:
                x1 = width - 1
                y1 = height - 1
                tile = img[y:height, x:width]
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
                #cv.imshow('tile', tile)

            elif y1 >= height:
                y1 = height - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
            
            elif x1 >= width:
                x1 = width - 1
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)
        
            else:
                cv.rectangle(img_copy, (x, y), (x1, y1), line_color, 1)

crop_grid(img, 2, 3, (0, 255, 0)) #run crop_grid function                
                
cv.imshow('patched image', img_copy)
cv.waitKey(0)
cv.destroyAllWindows()

2. Display image sequences of smooth transition of two images with different values of 
α
. Refer to code in section "Image blending". Use "lena.jfif" and "coins.jfif" as the base images.

In [4]:
img1 = cv.imread('lena.jfif')
img2 = cv.imread('coins.jfif')

new_shape = img1.shape[:2]
img2 = cv.resize(img2, new_shape)

#It will display blended images sequences (alpha = 0.0, 0.1, 0.2, ... , 0.9, 1.0)
for i in np.arange(0, 1.1, 0.1):
    bldimg = cv.addWeighted(img1, i, img2, 1 - i, 0)
    cv.imshow('blended_image alpha = ' + str(i), bldimg)

cv.waitKey(0)
cv.destroyAllWindows()

3. Rotate image by 45 degrees without cropping the sides of the image. (Hint: There are 2 strategies to tackle these problems). Use "lena.jfif" as the input image.
- Use external libraries imutils.
- Modify the transformation matrix.

In [6]:
#First way
import imutils
import math
img = cv.imread('lena.jfif')

rotated = imutils.rotate(img, angle=45)
cv.imshow("Rotated image", rotated)

cv.waitKey(0)
cv.destroyAllWindows()

In [7]:
#Second way
imgbd = cv.imread('lena.jfif')

y, x = imgbd.shape[:2]

p1 = np.array([x / 2, 0])
p2 = np.array([x / 2, y / 2])
p3 = np.array([0, y / 2])

pts1 = np.float32([p1, p2, p3]) # before transformation

#rotation
p1, p2, p3 = p1 - np.array([x / 2, y / 2]), p2 - np.array([x / 2, y / 2]), p3 - np.array([x / 2, y / 2])
rotation45 = np.array([[math.cos(-math.pi / 4), -math.sin(-math.pi / 4)], [math.sin(-math.pi / 4), math.cos(-math.pi / 4)]])
p1, p2, p3 = np.dot(rotation45, p1) + np.array([x / 2, y / 2]), np.dot(rotation45, p2) + np.array([x / 2, y / 2]), np.dot(rotation45, p3) + np.array([x / 2, y / 2])

pts2 = np.float32([p1, p2, p3]) # after transformation

M = cv.getAffineTransform(pts1,pts2)

rotated = cv.warpAffine(imgbd,M,(x, y))


cv.imshow("Rotated image", rotated)

cv.waitKey(0)
cv.destroyAllWindows()

4. Use the images with titles: "flower.jfif" and "native-bee.png". I want to put flower above an image. If I add two images, it will change color. If I blend it, I get a transparent effect. But I want it to be opaque. If it was a rectangular region, we could use the ROI as we did in the previous section. But flower is not a rectangular region. This is where bitwise operations, like AND, OR, NOT and XOR really come in handy. The associated functions are cv.bitwise_and(), cv.bitwise_or() and cv.bitwise_not(). You need to use cv.threshold function to segment the flower.

In [8]:
imgflo = cv.imread('flower.jfif') #img2
imgbee = cv.imread('native-bee.png') #img1

imgflo[120:183, :175] = (0, 0, 0) 

rows,cols,channels = imgflo.shape
roi = imgbee[0:rows, 0:cols]

imgflogray = cv.cvtColor(imgflo,cv.COLOR_BGR2GRAY)
ret, mask = cv.threshold(imgflogray, 10, 255, cv.THRESH_BINARY)
mask_inv = cv.bitwise_not(mask)

imgbee_bg = cv.bitwise_and(roi,roi,mask = mask_inv)

imgflo_fg = cv.bitwise_and(imgflo,imgflo,mask = mask)

dst = cv.add(imgbee_bg,imgflo_fg)
imgbee[0:rows, 0:cols] = dst
cv.imshow('res',imgbee)
cv.waitKey(0)
cv.destroyAllWindows()